Random forest preparation:
Below is the code I used for the IMD data for the FBA c/w - will be different as I am doing a classifiction rather than a regression and will use scikit rather than sklearn

Also, for the FBA, there were 7 input features (OSM info) and 1 output (IMD score). For this there will be up to 358 input features (spectral bands) and 1 output (PFT). Therefore will need a table with all of the info in - make by creating a vector file of the bsq? - test this.

Want to iterate through the data rather than adding manually (358 bands)

Then need to split into X (all the bands) and y (PFT)

When complete, try a range of proportions of training and test data and remember to cross validate results.

Also possibly use the k-means clustering to validate and compare results

In [102]:
import pandas as pd
import geopandas
cross_lochs = pd.read_csv('cross_lochs_sample_for_rf2.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf


,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT,line
0,617,0.000951,0.005150,0.005988,0.006720,0.007361,0.010606,0.012082,0.014234,0.015797,...,0.097626,0.096350,0.095506,0.098056,0.096859,618,POINT (444764.308 6472590.655),0.000951,calluna_mix,4
1,822,0.002991,0.004075,0.004819,0.005989,0.007388,0.009729,0.012035,0.013280,0.016029,...,0.091884,0.090096,0.090060,0.089917,0.090350,823,POINT (444774.308 6472585.655),0.002991,calluna_mix,5
2,1230,0.002812,0.003526,0.005374,0.006186,0.007536,0.009726,0.011938,0.013003,0.016773,...,0.087527,0.090578,0.089190,0.087724,0.090253,1231,POINT (444784.308 6472575.655),0.002812,calluna_mix,7
3,1841,0.002776,0.004215,0.004849,0.006305,0.008160,0.010193,0.012259,0.014886,0.015938,...,0.093998,0.096585,0.099090,0.097692,0.098034,1842,POINT (444794.308 6472560.655),0.002776,calluna_mix,10
4,2452,0.003292,0.003483,0.003686,0.005693,0.007814,0.009010,0.012300,0.013145,0.015174,...,0.094432,0.094272,0.097161,0.097323,0.095702,2453,POINT (444804.308 6472545.655),0.003292,calluna_mix,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,19455,0.000811,0.001748,0.002514,0.003086,0.002629,0.003547,0.005152,0.006167,0.008431,...,0.010550,0.012407,0.010226,0.010765,0.010614,19456,POINT (445574.308 6472130.655),0.000811,water,96
176,21481,0.000730,0.001129,0.001588,0.002542,0.002014,0.003243,0.004675,0.004775,0.006566,...,0.005632,0.006227,0.007418,0.005301,0.004491,21482,POINT (445554.308 6472080.655),0.000730,water,106
177,22294,0.000828,0.001481,0.002596,0.002506,0.002702,0.003921,0.005196,0.006644,0.008155,...,0.010491,0.014097,0.011939,0.011159,0.008924,22295,POINT (445559.308 6472060.655),0.000828,water,110
178,41439,0.000852,0.001206,0.003087,0.003662,0.005048,0.007325,0.009386,0.011490,0.013876,...,0.080829,0.080322,0.081170,0.080854,0.080922,41440,POINT (444859.308 6471585.655),0.000852,calluna_mix,205


In [103]:
from sklearn.model_selection import train_test_split

# train, test = train_test_split(cross_lochs_gdf, test_size=0.3)

# need to change so each PFT class is split 70:30
# potential long-winded method:extract all PFTs as separate groups and randomly split, 
# then join the test and train dataframes together

train_shrub_sphagnum, test_shrub_sphagnum = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('shrub_sphagnum')], test_size=0.3)
train_water, test_water = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('water')], test_size=0.3)
train_sphagnum_r, test_sphagnum_r = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('sphagnum_r')], test_size=0.3)
train_pool_bogbean, test_pool_bogbean = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('pool_bogbean')], test_size=0.3)
train_calluna_mix, test_calluna_mix = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('calluna_mix')], test_size=0.3)
train_rushes_sedges, test_rushes_sedges = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('rushes_sedges')], test_size=0.3)
# train_bare_water, test_bare_water = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('bare_water')], test_size=0.3)
# train_shrub_mix, test_shrub_mix = train_test_split(cross_lochs_gdf[cross_lochs_gdf['PFT'].str.contains('shrub_mix')], test_size=0.3)

train_frames = [train_shrub_sphagnum, train_water, train_sphagnum_r, train_pool_bogbean, train_calluna_mix, train_rushes_sedges]
test_frames = [test_shrub_sphagnum, test_water, test_sphagnum_r, test_pool_bogbean, test_calluna_mix, test_rushes_sedges]


train = pd.concat(train_frames)
test = pd.concat(test_frames)


train.head()

,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT,line
97,18562,0.001223,0.002153,0.004454,0.005516,0.007077,0.009220,0.011208,0.013527,0.015179,...,0.104686,0.106857,0.104221,0.104855,0.104162,18563,POINT (445169.308 6472150.655),0.001223,shrub_sphagnum,92
112,34835,0.000631,0.001842,0.004200,0.005745,0.007268,0.009819,0.011818,0.014151,0.016375,...,0.098929,0.101719,0.100788,0.102349,0.100488,34836,POINT (445334.308 6471750.655),0.000631,shrub_sphagnum,172
95,13869,0.001928,0.002206,0.004092,0.005457,0.006180,0.008672,0.010540,0.012283,0.014258,...,0.087433,0.087913,0.090433,0.087537,0.085878,13870,POINT (445049.308 6472265.655),0.001928,shrub_sphagnum,69
100,20407,0.001370,0.002398,0.005142,0.005869,0.007883,0.010136,0.012374,0.014006,0.016168,...,0.093195,0.094512,0.092757,0.090465,0.092138,20408,POINT (445259.308 6472105.655),0.001370,shrub_sphagnum,101
113,35853,0.000354,0.001751,0.004238,0.005341,0.006616,0.009189,0.010890,0.012489,0.014805,...,0.081865,0.082495,0.082351,0.080686,0.080357,35854,POINT (445349.308 6471725.655),0.000354,shrub_sphagnum,177


In [104]:
x_train = train[train.columns[1:359]]
y_train = train[train.columns[362]]

x_test = test[test.columns[1:359]]
y_test = test[test.columns[362]]

x_test.head(2)

,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
102,0.001359,0.002590,0.004418,0.006452,0.008141,0.010066,0.012321,0.014902,0.016941,0.019216,...,0.094812,0.095517,0.093724,0.093595,0.093233,0.095329,0.094489,0.095126,0.093151,0.093714
115,0.000281,0.001482,0.003366,0.005031,0.006199,0.008613,0.011078,0.013709,0.015994,0.017653,...,0.102098,0.104798,0.100608,0.101541,0.100636,0.102058,0.102319,0.102159,0.100453,0.103964


In [105]:
from sklearn import dummy

dc = dummy.DummyClassifier()
dc.fit(x_train, y_train)
dc_score = dc.score(x_test, y_test)

print("Baseline Accuracy:", dc_score)


Baseline Accuracy: 0.16666666666666666


In [115]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


clf = RandomForestClassifier(max_depth=2, random_state=42)
# fit to the training data
clf.fit(x_train, y_train)
print(clf.score(x_train, y_train))

# print(clf.predict(x_test))


# predict using the model
clf_train_pred = clf.predict(x_train)
clf_pred = clf.predict(x_test)

# append to the results dataframe
# model_results = model_results.assign(rf_pred = rf_pred)
# print(model_results)

# test the performance of the model
scores = cross_val_score(clf, x_train, y_train, cv=3)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.7380952380952381
0.60 accuracy with a standard deviation of 0.04


In [116]:
test_score = cross_val_score(clf, x_test, y_test, cv=3)
print("%0.2f accuracy with a standard deviation of %0.2f" % (test_score.mean(), test_score.std()))

0.61 accuracy with a standard deviation of 0.05


In [96]:
model_results = test[test.columns[360:363]]

model_results.head()

,geometry,GRID_CODE,PFT
97,POINT (445169.308 6472150.655),0.001223,shrub_sphagnum
112,POINT (445334.308 6471750.655),0.000631,shrub_sphagnum
98,POINT (445189.308 6472145.655),0.000968,shrub_sphagnum
109,POINT (445159.308 6471835.655),0.000772,shrub_sphagnum
94,POINT (445134.308 6472280.655),0.001527,shrub_sphagnum


In [97]:
model_results = model_results.assign(clf_pred = clf_pred)

model_results

,geometry,GRID_CODE,PFT,clf_pred
97,POINT (445169.308 6472150.655),0.001223,shrub_sphagnum,calluna_mix
112,POINT (445334.308 6471750.655),0.000631,shrub_sphagnum,shrub_sphagnum
98,POINT (445189.308 6472145.655),0.000968,shrub_sphagnum,pool_bogbean
109,POINT (445159.308 6471835.655),0.000772,shrub_sphagnum,calluna_mix
94,POINT (445134.308 6472280.655),0.001527,shrub_sphagnum,calluna_mix
89,POINT (444954.308 6472375.655),0.001433,shrub_sphagnum,calluna_mix
108,POINT (445489.308 6471950.655),0.000941,shrub_sphagnum,calluna_mix
107,POINT (445009.308 6471975.655),0.000197,shrub_sphagnum,pool_bogbean
104,POINT (445009.308 6472000.655),0.003219,shrub_sphagnum,shrub_sphagnum
173,POINT (445404.308 6472165.655),0.000791,water,water


In [98]:
train_focused = train[train.columns[360:363]]

train_focused.head()

,geometry,GRID_CODE,PFT
93,POINT (445114.308 6472290.655),0.001495,shrub_sphagnum
113,POINT (445349.308 6471725.655),0.000354,shrub_sphagnum
111,POINT (445004.308 6471775.655),0.000855,shrub_sphagnum
115,POINT (445134.308 6471655.655),0.000281,shrub_sphagnum
116,POINT (445029.308 6471650.655),0.000705,shrub_sphagnum


In [99]:
model_results.to_file("cross_lochs_results_sample_gdf.shp")

In [100]:
train_focused.to_file("cross_lochs_train_sample_gdf.shp")

In [101]:
model_results.to_csv("cross_lochs_results_sample_gdf.csv")
train_focused.to_csv("cross_lochs_train_sample_gdf.csv")

In [117]:
# now want to predict for the whole dataset using the random forest
cross_lochs = pd.read_csv('../cross_lochs_gdf.csv')

cross_lochs['geometry'] = geopandas.GeoSeries.from_wkt(cross_lochs['geometry'])

cross_lochs_gdf = geopandas.GeoDataFrame(cross_lochs, geometry='geometry')

cross_lochs_gdf

,Unnamed: 0,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,band353,band354,band355,band356,band357,band358,POINTID,geometry,GRID_CODE,PFT
0,0,0.001099,0.003430,0.004752,0.006146,0.006639,0.009240,0.010558,0.011710,0.013801,...,0.074587,0.072792,0.075150,0.074088,0.074698,0.072248,1,POINT (444724.308 6472605.655),0.001099,NaN
1,1,0.003365,0.002541,0.004668,0.006236,0.006519,0.008654,0.010603,0.012083,0.012975,...,0.069210,0.070987,0.072956,0.070970,0.067680,0.067113,2,POINT (444729.308 6472605.655),0.003365,NaN
2,2,0.002603,0.002340,0.005392,0.006031,0.006805,0.008977,0.010990,0.012230,0.014469,...,0.086315,0.086899,0.087825,0.089891,0.086141,0.085678,3,POINT (444734.308 6472605.655),0.002603,NaN
3,3,0.002381,0.004279,0.005815,0.006092,0.007620,0.009825,0.012374,0.014953,0.016326,...,0.103626,0.101852,0.101284,0.103357,0.100732,0.099883,4,POINT (444739.308 6472605.655),0.002381,NaN
4,4,0.002342,0.002837,0.005055,0.005912,0.006696,0.009762,0.010999,0.012902,0.015368,...,0.093658,0.094130,0.091242,0.095368,0.093193,0.093215,5,POINT (444744.308 6472605.655),0.002342,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42016,42016,0.001845,0.002345,0.004714,0.005430,0.006393,0.008860,0.010772,0.012449,0.014887,...,0.077407,0.076553,0.078615,0.078627,0.078251,0.077265,42017,POINT (445714.308 6471575.655),0.001845,NaN
42017,42017,0.002455,0.002271,0.004029,0.005480,0.006396,0.008863,0.011025,0.012685,0.014688,...,0.073803,0.074018,0.074871,0.075158,0.073604,0.075263,42018,POINT (445719.308 6471575.655),0.002455,NaN
42018,42018,0.002387,0.002279,0.004424,0.005426,0.006352,0.008929,0.010853,0.012708,0.014964,...,0.082753,0.083324,0.083870,0.084588,0.081337,0.082178,42019,POINT (445724.308 6471575.655),0.002387,NaN
42019,42019,0.002777,0.002263,0.004564,0.005324,0.006339,0.008972,0.011173,0.012952,0.015367,...,0.094481,0.092089,0.094319,0.095786,0.090843,0.091052,42020,POINT (445729.308 6471575.655),0.002777,NaN


In [121]:
x = cross_lochs_gdf[cross_lochs_gdf.columns[1:359]]

x.head()

,band1,band2,band3,band4,band5,band6,band7,band8,band9,band10,...,band349,band350,band351,band352,band353,band354,band355,band356,band357,band358
0,0.001099,0.003430,0.004752,0.006146,0.006639,0.009240,0.010558,0.011710,0.013801,0.015441,...,0.077946,0.077413,0.077153,0.078934,0.074587,0.072792,0.075150,0.074088,0.074698,0.072248
1,0.003365,0.002541,0.004668,0.006236,0.006519,0.008654,0.010603,0.012083,0.012975,0.015468,...,0.075149,0.073557,0.071610,0.071727,0.069210,0.070987,0.072956,0.070970,0.067680,0.067113
2,0.002603,0.002340,0.005392,0.006031,0.006805,0.008977,0.010990,0.012230,0.014469,0.017382,...,0.088601,0.087386,0.087973,0.089698,0.086315,0.086899,0.087825,0.089891,0.086141,0.085678
3,0.002381,0.004279,0.005815,0.006092,0.007620,0.009825,0.012374,0.014953,0.016326,0.018532,...,0.105301,0.103799,0.104660,0.107808,0.103626,0.101852,0.101284,0.103357,0.100732,0.099883
4,0.002342,0.002837,0.005055,0.005912,0.006696,0.009762,0.010999,0.012902,0.015368,0.017144,...,0.095351,0.096855,0.094500,0.093532,0.093658,0.094130,0.091242,0.095368,0.093193,0.093215


In [122]:
clf_pred_full = clf.predict(x)

clf_pred_full

array(['sphagnum_r', 'sphagnum_r', 'sphagnum_r', ..., 'calluna_mix',
       'calluna_mix', 'calluna_mix'], dtype=object)

In [125]:
model_results_full = cross_lochs_gdf[cross_lochs_gdf.columns[360:362]]

model_results_full = model_results_full.assign(clf_pred_full = clf_pred_full)

model_results_full.head()

,geometry,GRID_CODE,clf_pred_full
0,POINT (444724.308 6472605.655),0.001099,sphagnum_r
1,POINT (444729.308 6472605.655),0.003365,sphagnum_r
2,POINT (444734.308 6472605.655),0.002603,sphagnum_r
3,POINT (444739.308 6472605.655),0.002381,sphagnum_r
4,POINT (444744.308 6472605.655),0.002342,sphagnum_r


In [127]:
model_results_full.to_file("cross_lochs_results_sample_full_gdf.shp")

model_results_full.to_csv("cross_lochs_results_sample_full_gdf.csv")


/tmp/ipykernel_620/3141684826.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  model_results_full.to_file("cross_lochs_results_sample_full_gdf.shp")
